#  Stock Price Monitor

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import datetime
import time

In [2]:
url = "https://www.boerse.de/indizes/MSCI-World/XC0009692739"

In [3]:
html = urlopen(url)   
soup = bs(html.read())
soup = soup.find_all('span',{'class':'BW_PUSH'})

In [4]:
temp = []
for i in range(0, len(soup)):
    temp.append(str(soup[i]))
soup = temp

In [5]:
# "last", "perfInstAbs", "perfInstRel"

substrings = ["last", "perfInstAbs", "perfInstRel"]

for j in range(0, len(substrings)):
    temp_ind = []
    for i in range(0, len(soup)):
        if substrings[j] in soup[i]:
            temp_ind.append(i)
    print(temp_ind)
        

[24, 29, 32]
[25, 30, 33]
[27, 31, 34]


In [6]:
price_string = soup[29]
print(price_string)

<span class="BW_PUSH">
<span class="" data-format-option="2" data-push-agse="226974@173" data-push-attribute="last">
                                        2.401,84                                     </span>
</span>


In [7]:
da_string = soup[30]
print(da_string)

<span class="BW_PUSH">
<span class="" data-format-option="2" data-push-agse="226974@173" data-push-attribute="perfInstAbs">
                                        15,43                                     </span>
</span>


In [8]:
dr_string = soup[31]
print(dr_string)

<span class="BW_PUSH">
<span class="" data-push-agse="226974@173" data-push-attribute="perfInstRel" data-sign=" %">
                                        0,65 %
                                    </span>
</span>


In [9]:
price = re.findall("\d*\.*\d*\,\d*", price_string)
price = price[0]
print(price)

price = re.findall("\d*", price)
print(price)

price = float(str(price[0] + price[2] + "." + price[4]))
print(price)

2.401,84
['2', '', '401', '', '84', '']
2401.84


In [10]:
da = re.findall("\d*\,\d*", da_string)
da = da[0]
print(da)

da =re.findall("\-*\d*", da)
print(da)

da = float(str(da[0] + "." + da[2]))
print(da)

15,43
['15', '', '43', '']
15.43


In [11]:
dr = re.findall("\-*\d*\,\d*", dr_string)
dr = dr[0]
print(dr)

dr =re.findall("\-*\d*", dr)
print(dr)

dr = float(str(dr[0] + "." + dr[2]))
print(dr)

0,65
['0', '', '65', '']
0.65


In [12]:
print("MSCI World Index stock is at ", price, "$ at ", datetime.datetime.now(), ".")
print("The absolute 24h development is ", da, ".")
print("The relative 24h development is ", dr, "% .")

MSCI World Index stock is at  2401.84 $ at  2020-01-16 21:37:37.642690 .
The absolute 24h development is  15.43 .
The relative 24h development is  0.65 % .


In [13]:
def stock_monitor(url, name, minutes):
    
    minute = 1
    while minute <= minutes: 
        
        html = urlopen(url)   
        soup = bs(html.read())
        soup = soup.find_all('span',{'class':'BW_PUSH'}) 
    
        temp = []
        for i in range(0, len(soup)):
            temp.append(str(soup[i]))
        soup = temp
    
        price_string = soup[29]
        da_string = soup[30]
        dr_string = soup[31]
    
        price = re.findall("\d*\.*\d*\,\d*", price_string)
        price = price[0]
        price = re.findall("\d*", price)
        price = float(str(price[0] + price[2] + "." + price[4]))
    
        da = re.findall("\-*\d*\,\d*", da_string)
        da = da[0]
        da = re.findall("\-*\d*", da)
        da = float(str(da[0] + "." + da[2]))
    
        dr = re.findall("\-*\d*\,\d*", dr_string)
        dr = dr[0]
        dr =re.findall("\-*\d*", dr)
        dr = float(str(dr[0] + "." + dr[2]))
    
        print(name ,"stock is at ", price, "$ at ", datetime.datetime.now(), ".")
        print("The absolute 24h development is ", da, ".")
        print("The relative 24h development is ", dr, "% .")
        print(" ")
        
        time.sleep(60)
        minute += 1
        
        
      

In [14]:
url = "https://www.boerse.de/indizes/MSCI-World/XC0009692739"
name = "MSCI WORLD INDEX"

stock_monitor(url, name, minutes=5)

MSCI WORLD INDEX stock is at  2401.84 $ at  2020-01-16 21:37:38.854501 .
The absolute 24h development is  15.43 .
The relative 24h development is  0.65 % .
 
MSCI WORLD INDEX stock is at  2401.91 $ at  2020-01-16 21:38:40.286877 .
The absolute 24h development is  15.5 .
The relative 24h development is  0.65 % .
 
MSCI WORLD INDEX stock is at  2403.1 $ at  2020-01-16 21:39:43.930109 .
The absolute 24h development is  16.69 .
The relative 24h development is  0.7 % .
 
MSCI WORLD INDEX stock is at  2403.14 $ at  2020-01-16 21:40:46.792393 .
The absolute 24h development is  16.74 .
The relative 24h development is  0.7 % .
 
MSCI WORLD INDEX stock is at  2403.15 $ at  2020-01-16 21:41:48.446996 .
The absolute 24h development is  16.74 .
The relative 24h development is  0.7 % .
 


In [15]:
url = "https://www.boerse.de/indizes/Dax/DE0008469008"
name = "DAX"

stock_monitor(url, name, minutes=5)

DAX stock is at  13460.5 $ at  2020-01-16 21:42:50.000418 .
The absolute 24h development is  28.2 .
The relative 24h development is  0.21 % .
 
DAX stock is at  13461.02 $ at  2020-01-16 21:43:51.296129 .
The absolute 24h development is  28.98 .
The relative 24h development is  0.22 % .
 
DAX stock is at  13461.5 $ at  2020-01-16 21:44:52.828621 .
The absolute 24h development is  29.2 .
The relative 24h development is  0.22 % .
 
DAX stock is at  13458.0 $ at  2020-01-16 21:45:55.129427 .
The absolute 24h development is  31.5 .
The relative 24h development is  0.23 % .
 
DAX stock is at  13460.54 $ at  2020-01-16 21:47:02.265805 .
The absolute 24h development is  28.49 .
The relative 24h development is  0.21 % .
 


In [16]:
url = "https://www.boerse.de/rohstoffe/Goldpreis/XC0009655157"
name = "GOLD"

stock_monitor(url, name, minutes=5)

GOLD stock is at  1552.87 $ at  2020-01-16 21:48:03.572719 .
The absolute 24h development is  -3.35 .
The relative 24h development is  -0.22 % .
 
GOLD stock is at  1553.14 $ at  2020-01-16 21:49:04.998422 .
The absolute 24h development is  -3.4 .
The relative 24h development is  -0.22 % .
 
GOLD stock is at  1553.29 $ at  2020-01-16 21:50:06.595570 .
The absolute 24h development is  -3.24 .
The relative 24h development is  -0.21 % .
 
GOLD stock is at  1553.28 $ at  2020-01-16 21:51:08.059902 .
The absolute 24h development is  -3.26 .
The relative 24h development is  -0.21 % .
 
GOLD stock is at  1553.41 $ at  2020-01-16 21:52:11.410043 .
The absolute 24h development is  -2.81 .
The relative 24h development is  -0.18 % .
 


In [17]:
url = "https://www.boerse.de/indizes/Dow-Jones/US2605661048"
name = "DOW JONES"

stock_monitor(url, name, minutes=5)

DOW JONES stock is at  29282.23 $ at  2020-01-16 21:53:13.615230 .
The absolute 24h development is  243.83 .
The relative 24h development is  0.84 % .
 
DOW JONES stock is at  29285.35 $ at  2020-01-16 21:54:14.840900 .
The absolute 24h development is  246.95 .
The relative 24h development is  0.85 % .
 
DOW JONES stock is at  29283.67 $ at  2020-01-16 21:55:17.757534 .
The absolute 24h development is  245.27 .
The relative 24h development is  0.84 % .
 
DOW JONES stock is at  29284.2 $ at  2020-01-16 21:56:20.902699 .
The absolute 24h development is  245.8 .
The relative 24h development is  0.85 % .
 
DOW JONES stock is at  29252.0 $ at  2020-01-16 21:57:23.875260 .
The absolute 24h development is  200.0 .
The relative 24h development is  0.69 % .
 
